In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

import datetime
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

In [2]:
train = pd.read_pickle("data/train.p")
test = pd.read_pickle("data/test.p")
train.keys()

X_train, y_train = train['features'], train['labels']
X_train.shape, y_train.shape

X_test, y_test = test['features'], test['labels']
X_test.shape, y_test.shape

signs = pd.read_csv('data/signnames.csv')

dict_signs = signs.to_dict()['b']

In [3]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [4]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [5]:
def get_cnn_v1(input_shape, num_classes):
    model = Sequential([
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        Flatten(),
        Dense(num_classes, activation='softmax')
    ])
    
    return model

def train_model(model, X_train, y_train, params_fit={}):    
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
    
    model.fit(
        X_train,
        y_train,
        batch_size=params_fit.get('batch_size', 128),
        epochs=params_fit.get('epochs', 5),
        verbose=params_fit.get('verbose', 1),
        validation_data=params_fit.get('validation_data', (X_train, y_train)),
        callbacks = [tensorboard_callback]
    )
    
    return logdir
    

In [6]:
model = get_cnn_v1(input_shape, num_classes)
logdir = train_model(model, X_train, y_train)

Train on 34799 samples, validate on 34799 samples
Epoch 1/5
10624/34799 [========>.....................] - ETA: 22s - loss: 179.2406 - accuracy: 0.4021

KeyboardInterrupt: 

In [ ]:
%tensorboard --logdir logs

In [9]:
def get_cnn_v2(input_shape, num_classes):
    model = Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPool2D(),
        Dropout(0.5),
        
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPool2D(),
        Dropout(0.3),
        
        Flatten(),
        
        Dense(1024, activation='relu'),
        Dropout(0.5),
        
        Dense(num_classes, activation='softmax')
    ])
    
    return model

In [10]:
model2 = get_cnn_v2(input_shape, num_classes)

In [11]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)             

In [12]:
logdir2 = train_model(model2, X_train, y_train)

Train on 34799 samples, validate on 34799 samples
Epoch 1/5
34799/34799 [==============================] - 61s 2ms/sample - loss: 6.2611 - accuracy: 0.1847 - val_loss: 2.1625 - val_accuracy: 0.4481
Epoch 2/5
34799/34799 [==============================] - 61s 2ms/sample - loss: 1.9137 - accuracy: 0.4495 - val_loss: 1.1886 - val_accuracy: 0.6494
Epoch 3/5
34799/34799 [==============================] - 59s 2ms/sample - loss: 1.1536 - accuracy: 0.6446 - val_loss: 0.5376 - val_accuracy: 0.8659
Epoch 4/5
34799/34799 [==============================] - 58s 2ms/sample - loss: 0.7650 - accuracy: 0.7619 - val_loss: 0.3043 - val_accuracy: 0.9297
Epoch 5/5
34799/34799 [==============================] - 59s 2ms/sample - loss: 0.5938 - accuracy: 0.8151 - val_loss: 0.2073 - val_accuracy: 0.9549


In [18]:
%tensorboard --logdir "logs"

Reusing TensorBoard on port 6006 (pid 1316), started 0:54:26 ago. (Use '!kill 1316' to kill it.)